In [1]:
from PIL import Image, PngImagePlugin
import hashlib
import json
from datetime import datetime
import os

import unibox as ub

gpt2 client:

In [2]:
# gpt2 api

from gradio_client import Client

client = Client("https://cca2aa87a2dbcc14d3.gradio.live/")
result = client.predict(
		"Hello!!",	# str in 'prompt' Textbox component
		10,	# float (numeric value between 10 and 300) in 'max_length' Slider component
		api_name="/predict"
)
print(result)

Loaded as API: https://cca2aa87a2dbcc14d3.gradio.live/ ✔
Hello!!, pokemon (creature), red hair, ope


webui:

In [ ]:
# https://github.com/troph-team/eval-it/blob/aa0cb59983e2b0385ef03328b2ce6a3c36a073a0/evalit/webui/webui_t2i_client.py#L38C7-L38C27
from evalit.webui.webui_t2i_client import WebuiT2iClient
from evalit.webui.webui_t2i_client import WebuiT2iClient, SdxlGenerationConfig
from evalit.webui.webui_options_manager import OptionsManager


def save_image(image_paths:list[str], param_strs:list[str], api_args, save_dir:str="saved_images"):
    """保存一个webui生成结果到本地 (可能包括多张图)

    """

    saved_files = []

    for i, (image, param) in enumerate(zip(image_paths, param_strs)):
        os.makedirs(save_dir, exist_ok=True)
        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", param)
        pnginfo.add_text("api_args", json.dumps(api_args))

        # get filename
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        img_hash = hashlib.md5(image.tobytes()).hexdigest()        
        file_name = f"{timestamp}_{img_hash[:4]}.png"        
        file_path = os.path.join(save_dir, file_name)

        # save file
        image.save(file_path, pnginfo=pnginfo)
        saved_files.append(file_path)  # Add the saved file path to the list

    return saved_files

def generate_image(prompt:str, save_dir:str="saved_images"):
    """从webui api roll图, 然后返回本地保存路径
    (假设模型已经在webui里手动调到需要的那个)
    """
    # initialize client
    client = WebuiT2iClient(baseurl="http://0.0.0.0:7862")

    # initialize options manager
    client_base_url = client.baseurl
    options_manager = OptionsManager(client_base_url)

    # https://github.com/troph-team/eval-it/blob/aa0cb59983e2b0385ef03328b2ce6a3c36a073a0/evalit/webui/webui_t2i_client.py#L38  
    config = SdxlGenerationConfig() 
    config.steps=20
    config.cfg_scale=6.5

    # generate images
    images, param_strs, api_args = client.generate(prompt, config)  # negative defined in config

    # save images
    saved_files = save_image(images, param_strs, api_args, save_dir) # list of paths to saved images
    return saved_files

# save_dir = "saved_images"
# img_path = generate_image("a cat")[0]
# display(ub.loads(img_path))

gradio: